References:  
GitHub  
https://github.com/PaddlePaddle/PaddleOCR  
  
Google Colaboratoryで開く  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/paddleOCR_CharacterDetection_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# Google Driveのマウント
Google ColaboratoryとGoogle Driveを接続します。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 環境セットアップ

## PaddlePaddleのインストール

In [ ]:
!python3 -m pip install paddlepaddle-gpu -i https://mirror.baidu.com/pypi/simple

## PaddlePaddleのインストール確認
PaddlePaddleのバージョンと認識しているGPUの数を出力

In [ ]:
!python3 -c "import paddle; print(paddle.__version__)"
!python3 -c "import paddle; paddle.utils.run_check()"

## PaddleOCRのインストール

In [ ]:
%cd /content/
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

## 関連するライブラリのインストール

In [ ]:
%cd /content/PaddleOCR
!pip install -r requirements.txt

# 文字検出


## テスト用画像のダウンロード

In [ ]:
%cd /content/PaddleOCR

!wget https://paddleocr.bj.bcebos.com/dygraph_v2.1/ppocr_img.zip
!unzip ppocr_img.zip > /dev/null

## 学習済みモデルのダウンロード

In [ ]:
%cd /content/PaddleOCR
!mkdir pretrain_models
%cd /content/PaddleOCR/pretrain_models

!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_det_infer.tar
!tar -xvf ch_ppocr_server_v2.0_det_infer.tar

## 文字検出

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [ ]:
%cd /content/PaddleOCR
!python3 tools/infer/predict_det.py \
  --det_algorithm="DB" \
  --det_model_dir="./pretrain_models/ch_ppocr_server_v2.0_det_infer" \
  --image_dir="./ppocr_img/imgs/12.jpg" \
  --use_gpu=True

## 検出結果の表示

In [ ]:
%cd /content/PaddleOCR
result_path = "./inference_results/det_res_12.jpg"

# 画像の表示
image = Image.open(result_path).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

# 文字検出モデルのトレーニング

## データ準備

In [ ]:
%cd /content/PaddleOCR

# 登録必須
# 学習データ(ICDAR2015)ダウンロード
!wget --no-check-certificate https://rrc.cvc.uab.es/downloads/ch4_training_images.zip
!wget --no-check-certificate https://rrc.cvc.uab.es/downloads/ch4_test_images.zip
!unzip ch4_training_images.zip -d ./icdar_c4_train_imgs > /dev/null
!unzip ch4_test_images.zip -d ./ch4_test_images > /dev/null

!wget https://paddleocr.bj.bcebos.com/dataset/train_icdar2015_label.txt
!wget https://paddleocr.bj.bcebos.com/dataset/test_icdar2015_label.txt

## 学習済みモデルのダウンロード

In [ ]:
%cd /content/PaddleOCR/pretrain_models

!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_server_v2.0_det_train.tar
!tar -xvf ch_ppocr_server_v2.0_det_train.tar

## model保存用ディレクトリ作成

In [ ]:
!mkdir -p /content/drive/MyDrive/Colab\ Notebooks/output/ch_db_res18/

## モデルのトレーニング

In [ ]:
%cd /content/PaddleOCR

# 学習途中のモデルを確認
ckpt_path = ""
# 最新エポックのモデルを確認
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/latest.pdopt"):
  ckpt_path = "/content/drive/MyDrive/Colab\ Notebooks/output/ch_db_res18/latest"
# eval後のモデルを確認。evalがあれば最新エポックのモデルではなくeval後を利用
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/best_accuracy.pdopt"):
  ckpt_path = "/content/drive/MyDrive/Colab\ Notebooks/output/ch_db_res18/best_accuracy"
if ckpt_path == "":
  print("There is no ckpt. Train a new model.")
else:
  print("Resume training from", ckpt_path)

#-o Global.pretrained_model="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/latest" \
!python3 tools/train.py \
  -c configs/det/det_res18_db_v2.0.yml \
  -o \
  Global.epoch_num=30 \
  Global.eval_batch_step=[725,125] \
  Global.pretrained_model="./pretrain_models/ch_ppocr_server_v2.0_det_train/best_accuracy" \
  Global.checkpoints={ckpt_path} \
  Global.save_model_dir="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/" \
  Global.save_res_path="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/" \
  Train.dataset.data_dir=./ \
  Train.dataset.label_file_list=[./train_icdar2015_label.txt] \
  Eval.dataset.data_dir=./ \
  Eval.dataset.label_file_list=[./test_icdar2015_label.txt] \

# モデルの評価

## precision, recall, hmeanの算出

In [ ]:
%cd /content/PaddleOCR

# latest以外にbest_accracyが出力されている場合はbest_accuracyを使用
ckpt_path = "/content/drive/MyDrive/Colab\ Notebooks/output/ch_db_res18/latest"
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/best_accuracy.pdopt"):
  ckpt_path = "/content/drive/MyDrive/Colab\ Notebooks/output/ch_db_res18/best_accuracy"
print("ckpt path", ckpt_path)

!python3 tools/eval.py \
  -c configs/det/det_res18_db_v2.0.yml \
  -o \
  Global.pretrained_model="./pretrain_models/ch_ppocr_server_v2.0_det_train/best_accuracy" \
  Global.checkpoints={ckpt_path} \
  Global.save_model_dir="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/" \
  Global.save_res_path="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/" \
  Train.dataset.data_dir=./ \
  Train.dataset.label_file_list=[./train_icdar2015_label.txt] \
  Eval.dataset.data_dir=./ \
  Eval.dataset.label_file_list=[./test_icdar2015_label.txt] \
  PostProcess.box_thresh=0.6 \
  PostProcess.unclip_ratio=1.5

## 学習用モデルを推論用モデルに変換

In [ ]:
!python3 tools/export_model.py \
  -c configs/det/det_res18_db_v2.0.yml \
  -o \
  Global.pretrained_model={ckpt_path} \
  Global.save_inference_dir="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/det_db_inference/"

## トレーニングしたモデルを使って文字検出

In [ ]:
%cd /content/PaddleOCR
!python3 tools/infer/predict_det.py \
  --det_algorithm="DB" \
  --det_model_dir="/content/drive/MyDrive/Colab Notebooks/output/ch_db_res18/det_db_inference" \
  --image_dir="./ch4_test_images/img_394.jpg" \
  --use_gpu=True

In [ ]:
%cd /content/PaddleOCR
result_path = "./inference_results/det_res_img_394.jpg"
!mv ./inference_results/det_res_img_394.jpg ./inference_results/train_det_res_img_394.jpg
result_path = "./inference_results/train_det_res_img_394.jpg"

# 画像の表示
image = Image.open(result_path).convert("RGB")
plt.figure(figsize=(6, 6))
plt.imshow(image)

## 配布されているモデルと比較

In [ ]:
%cd /content/PaddleOCR
!python3 tools/infer/predict_det.py \
  --det_algorithm="DB" \
  --det_model_dir="./pretrain_models/ch_ppocr_server_v2.0_det_infer" \
  --image_dir="./ch4_test_images/img_394.jpg" \
  --use_gpu=True

In [ ]:
%cd /content/PaddleOCR
pretrain_result_path = "./inference_results/det_res_img_394.jpg"

image = Image.open(result_path).convert("RGB")
pre_image = Image.open(pretrain_result_path).convert("RGB")

dst = Image.new('RGB', (image.width + pre_image.width, image.height))
dst.paste(image, (0, 0))
dst.paste(pre_image, (image.width, 0))

# 画像の表示
plt.figure(figsize=(12, 12))
plt.imshow(dst)